In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
import shelve
from sklearn import metrics
import time
from sklearn.externals import joblib
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, mutual_info_classif
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
start = time.time()

### 读取样本

In [2]:
shelve_file = shelve.open('../data/sample')
X_train = shelve_file['X_train']
X_train2 = X_train.copy()
y_train = shelve_file['y_train']
X_test = shelve_file['X_test']

### 将样本转化为向量

In [3]:

# tfidf = TfidfVectorizer(ngram_range=(1,4), min_df=3, max_df=0.9, use_idf=1, smooth_idf=1, sublinear_tf=1)
# tfidf.fit(X_train)
# X_train = tfidf.transform(X_train)
# #skb = SlectKBest(chi2,k = 500000)#特征选择
# #X_train = skb.fit_transform(X_train,y_train) 
# clf = SGDClassifier(loss="modified_huber", penalty="l2", alpha=0.00002, n_jobs=-1)#分类器
# print(X_train.shape)


(100562, 8514473)


/home/amax/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


### pipelie

In [4]:
sgd = ('clf',SGDClassifier(loss="modified_huber", alpha=0.00002, n_jobs=-1,penalty = 'l2'))
lsvc = ('clf',LinearSVC(C=1.5, loss='squared_hinge', intercept_scaling=1, tol=0.0002, multi_class='ovr'))
rfc = ('clf', RandomForestClassifier(n_estimators=50,max_features='auto'))  
knn = ('clf', KNeighborsClassifier())
svc = ('clf', SVC(kernel='linear', C=1.5))
mnb = ('clf',MultinomialNB(alpha = 0.00005))
lr = ('clf',LogisticRegression(penalty = 'l2',C=100))
dtc = ('clf', DecisionTreeClassifier(max_features='auto')) 
gdbt = ('clf', GradientBoostingClassifier()) 
tfidf = ('tfidf',TfidfVectorizer(ngram_range=(1,5), min_df=3, max_df=0.9, norm='l2', sublinear_tf=1))
#count = ('count',CountVectorizer(ngram_range=(1,4), min_df=3, max_df=0.9))
steps = [tfidf,
        (('skb'),SelectKBest(chi2,k =7000000)),
        lsvc]
clf = Pipeline(steps)



/home/amax/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:84: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


### 训练&存储

In [5]:
train_start = time.time()
clf.fit(X_train, y_train) #训练
print("Time consumption on train",time.time( ) - train_start)
'''
save_start = time.time()
joblib.dump(clf, 'model/lsvc.m', compress=3)#保存模型
print("Time consumption on save",time.time( ) - save_start)
'''


AttributeError: lower not found

### 训练集效果

In [ ]:

prediction = clf.predict(X_train)
print(metrics.classification_report(y_train,prediction))#分类报告
print(metrics.confusion_matrix(y_train,prediction))#混淆矩阵


### 预测

In [ ]:
test_start = time.time()
#预测
prediction = clf.predict(X_test)
print('Time comsumption on test',time.time()-test_start)
#保存结果
f_out = open('ans.csv', 'w')
f_out.write("id,class"+"\n")
for i in range(len(X_test)):
    f_out.write(str(i)+","+str(prediction[i])+'\n')
f_out.close()
print('Time comsumption:',(time.time()-start)/3600,'h')

sgd:
mf = 200w 0.76464
mf = 300w 0.7643
mf = 400w 0.76466
mf = 500w 0.7647
mf = 600w 0.7636
mf = 700w 0.7645
mf = 800w 0.7643
lsvc:
mf = 800w 0.7648
mf = 700w 0.7648
mf = 600w 0.7648
mf = 500w 0.7648
mf = 400w 0.7647
mf = 300w 0.7650
rfc:
0.5167
mnb:
1.2 0.4707
0.02 0.7061
0.001 0.7185
0.0005 0.7172
0.0001 0.7132
0.00005 0.7111
dtc:
0.4989
knn:could not convert integer scalar
lr:

1.0 0.7339
10 0.7606
20 0.7619
30 0.7625
40 0.7629
50 0.7633
60 0.7632
70 0.7641
80 0.7630
90 0.7631
100 0.7631